# Gradient Boosting et XGBoost

Le **boosting** est une technique de réduction du biais et de la variance utilisé en appretissage supervisé. L'idée sous-jacente est de construire un modèle complexe à partir de plusieurs modèles simples, dits *faibles*.

Considérons un classifieur faible initial $f_0$. Après l'avoir optimisé, la méthode de boosting cherche à construire un nouveau classifieur faible $f_1$ à partir de $f_0$ en introduction un terme *de résidu* $h$ :
$$f_1(x)=f_0(x) + h(x)$$
De sorte que $f_1$ soit *plus performant* que $f_0$. On répétant l'opération un certain nombre de fois, disons $p$, on construit un classifieur final $F$ complexe qui est une combinaison linéaire des $f_i$, où chacun des $f_i$ est associé à un poids $\alpha_i$ :

$$F(x)=\sum_{i=1}^n \alpha_i f_i(x)$$

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import scipy
import shap

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

import xgboost as xgb

In [2]:
prices = pd.read_csv("/home/maxime/Documents/Hackademy/Codes Formations/Datagram_AirBnB/data/price_availability.csv", sep=";")
listings = pd.read_csv("../clean/listings_final.csv", sep=";")
listings = listings.drop(589)  # Cet individu n'a pas de prix !
print("Chargement des données effectué.")

Chargement des données effectué.


In [3]:
uniques = np.unique(prices["listing_id"]) 
intersects = np.intersect1d(uniques, np.unique(listings["listing_id"]))
listings_prices = prices[prices["listing_id"].isin(intersects)]

In [4]:
dates =  pd.to_datetime(listings_prices["day"], format='%Y-%m-%d')
week_prices = listings_prices.assign(date=dates).set_index(dates).groupby(['listing_id', pd.Grouper(key='date', freq='7D')]).mean().reset_index().loc[:, ["listing_id", "date", "local_price"]]
week_prices.head()

,listing_id,date,local_price
0,56093,2018-08-27,170.0
1,56093,2018-09-03,170.0
2,56093,2018-09-10,170.0
3,56093,2018-09-17,170.0
4,56093,2018-09-24,170.0


In [13]:
Y = []

for i, row in listings.iterrows():
    y = 0
    ID = int(row["listing_id"])
    subset = listings_prices[listings_prices["listing_id"] == ID]
    y = subset["local_price"].mean()
    Y.append(y)

In [58]:
X_train, X_test, Y_train, Y_test = train_test_split(
    listings.loc[:, ["latitude", "longitude", "is_fully_refundable", "person_capacity", "bedrooms", "bathrooms"]],
    np.asarray(Y),
    test_size=0.2, random_state=10)

In [59]:
dtrain = xgb.DMatrix(X_train, Y_train)
dtest = xgb.DMatrix(X_test, Y_test)

In [60]:
param = {'max_depth':3, 'eta':0.1, 'silent':1, 'objective':'reg:linear' }
num_round = 100
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)

In [61]:
print('Variance score: %.4f' % r2_score(Y_test, preds))

Variance score: 0.5107
